In [34]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras
from keras import callbacks

from collections import defaultdict

import tensorflow as tf
from src.learner.game_to_features import FeatureSet_v1_1
from src.core.game_record import GameRecord
from src.core.board import Board

In [14]:
!gsutil ls gs://itd-aia-ds-dproc-staging/q-gomoku/

gs://itd-aia-ds-dproc-staging/q-gomoku/
gs://itd-aia-ds-dproc-staging/q-gomoku/.gitignore
gs://itd-aia-ds-dproc-staging/q-gomoku/LICENSE
gs://itd-aia-ds-dproc-staging/q-gomoku/Makefile
gs://itd-aia-ds-dproc-staging/q-gomoku/README
gs://itd-aia-ds-dproc-staging/q-gomoku/__init__.py
gs://itd-aia-ds-dproc-staging/q-gomoku/environment.yml
gs://itd-aia-ds-dproc-staging/q-gomoku/fs_wait_train.py
gs://itd-aia-ds-dproc-staging/q-gomoku/.git/
gs://itd-aia-ds-dproc-staging/q-gomoku/.idea/
gs://itd-aia-ds-dproc-staging/q-gomoku/conf/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-00-00-00/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-00:00:00/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-06-37-51/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-06-43-09/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-16-05-49/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-21-30/
gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-20-21-31/
gs://itd-aia-ds-dproc-

In [15]:
def make_features(row):
    gamestring = row.gamestring
    fs = FeatureSet_v1_1(gamestring)
    return zip(fs.q_features, fs.q_labels, fs.p_features, fs.p_labels)

gamestrings = spark.read.parquet('gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-22-18-51/').rdd.map(lambda x : x.gamestring).collect()

#q_features, q_labels, p_features, p_labels =\
#    map(list, zip(*spark.read.parquet('gs://itd-aia-ds-dproc-staging/q-gomoku/games2019-06-22-08-18/').repartition(10) \
#    .rdd.flatMap(make_features))).collect()

In [21]:
step = 2

In [38]:
q_features = []
q_labels = []
p_features = []
p_labels = []
game_records = []
for gamestring in gamestrings:
    fs = FeatureSet_v1_1(gamestring)
    game_records.append(GameRecord.parse(gamestring))
    q_features.extend(fs.q_features)
    q_labels.extend(fs.q_labels)
    p_features.extend(fs.p_features)
    p_labels.extend(fs.p_labels)

In [39]:
results = defaultdict(int)

for record in game_records:
    results[record.get_winning_player()] += 1
    
print("Games", len(gamestrings), "Results", results, "Q_features", len(q_features))

Games 500 Results defaultdict(<class 'int'>, {1: 260, 2: 240}) Q_features 67880


In [18]:
SIZE = q_features[0].shape[0]
CHANNELS = q_features[0].shape[2]

In [19]:
inp = Input(shape=(SIZE, SIZE, CHANNELS))

# key difference between this and conv network is padding
conv_1 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(inp)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(32, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(16, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn3)
bn4 = BatchNormalization()(conv_3)
conv_4 = Convolution2D(16, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn4)
bn5 = BatchNormalization()(conv_4)
conv_5 = Convolution2D(8, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn4)
bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn3)

hidden = Dense(10, activation='relu', kernel_initializer='random_normal', use_bias=False)(flat)
#bn_hidden = BatchNormalization()(hidden)
#hidden_2 = Dense(50, activation='relu', kernel_initializer='random_normal', use_bias=False)(bn_hidden)
bn_final = BatchNormalization()(hidden)

out = Dense(1, use_bias=False)(bn_final)

q_model = Model(inputs=[inp], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])


In [22]:
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
checkpoint = callbacks.ModelCheckpoint('../models/v2_' + str(step) + '_value.net', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=2)
callback_list = [es, checkpoint]

with tf.device('/gpu:0'):
    q_model.fit(x=np.array(q_features), 
                y=q_labels,
                callbacks=callback_list,
                shuffle=True,
                epochs=100,
                verbose=1,
                batch_size=500,
                validation_split=0.1)

Train on 61092 samples, validate on 6788 samples
Epoch 1/100
61092/61092 [==============================] - 2s 32us/step - loss: 0.2014 - mean_squared_error: 0.2014 - val_loss: 0.1014 - val_mean_squared_error: 0.1014
Epoch 2/100
61092/61092 [==============================] - 1s 15us/step - loss: 0.0299 - mean_squared_error: 0.0299 - val_loss: 0.0467 - val_mean_squared_error: 0.0467

Epoch 00002: val_loss improved from inf to 0.04673, saving model to ../models/v2_2_value.net
Epoch 3/100
61092/61092 [==============================] - 1s 15us/step - loss: 0.0235 - mean_squared_error: 0.0235 - val_loss: 0.0394 - val_mean_squared_error: 0.0394
Epoch 4/100
61092/61092 [==============================] - 1s 15us/step - loss: 0.0217 - mean_squared_error: 0.0217 - val_loss: 0.0405 - val_mean_squared_error: 0.0405

Epoch 00004: val_loss improved from 0.04673 to 0.04048, saving model to ../models/v2_2_value.net
Epoch 5/100
61092/61092 [==============================] - 1s 15us/step - loss: 0.0201 

In [23]:
inp = Input(shape=(SIZE, SIZE, CHANNELS))

conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(inp)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn3)
bn4 = BatchNormalization()(conv_3)
conv_4 = Convolution2D(32, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn4)
bn5 = BatchNormalization()(conv_4)
conv_5 = Convolution2D(16, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn5)
bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn6)

hidden = Dense(SIZE ** 2, activation='relu', kernel_initializer='random_normal', use_bias=False)(flat)
bn_final = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn_final)

p_model = Model(inputs=[inp], outputs=out)
p_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
checkpoint = callbacks.ModelCheckpoint('../models/v2_' + str(step) + '_policy.net', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=2)
callback_list = [es, checkpoint]

with tf.device('/gpu:0'):
    p_model.fit(x=np.array(p_features), 
                y=np.array(p_labels),
                callbacks=callback_list,
                shuffle=True,
                epochs=100,
                verbose=1,
                batch_size=500,
                validation_split=0.1)

Train on 61092 samples, validate on 6788 samples
Epoch 1/100
61092/61092 [==============================] - 3s 53us/step - loss: 4.3136 - acc: 0.0379 - val_loss: 4.2392 - val_acc: 0.0439
Epoch 2/100
61092/61092 [==============================] - 1s 24us/step - loss: 4.0901 - acc: 0.0637 - val_loss: 4.2300 - val_acc: 0.0560

Epoch 00002: val_loss improved from inf to 4.23001, saving model to ../models/v2_2_policy.net
Epoch 3/100
61092/61092 [==============================] - 1s 24us/step - loss: 3.9245 - acc: 0.0900 - val_loss: 4.1683 - val_acc: 0.0603
Epoch 4/100
61092/61092 [==============================] - 1s 24us/step - loss: 3.7512 - acc: 0.1169 - val_loss: 4.2128 - val_acc: 0.0601

Epoch 00004: val_loss improved from 4.23001 to 4.21277, saving model to ../models/v2_2_policy.net
Epoch 5/100
61092/61092 [==============================] - 1s 24us/step - loss: 3.5684 - acc: 0.1493 - val_loss: 4.2473 - val_acc: 0.0607
Epoch 00005: early stopping
